In [1]:
# default_exp config
# export
from pathlib import Path
import yaml
import os
import time

In [13]:
# export
class Config():
    config_file = Path(os.getenv('QS_CONFIG', f'~/.config/qs.yaml')).expanduser() 

    def __init__(self):
        self.config_file.parent.mkdir(parents=True, exist_ok=True)
        if not self.config_file.exists(): self.create_config()
        self.d = self.load_config()
           
    def __getattr__(self, k):
        if k=='d': raise AttributeError
        return self.d[k]
                       
    def create_config(self):
        config = {'version': 1.0, 'projects': {}}
        self.save_config(config)
                       
    def load_config(self):
        with self.config_file.open() as f:
            return yaml.load(f, Loader=yaml.FullLoader)
                       
    def save_config(self, config):
        with self.config_file.open('w') as f:
            yaml.dump(config, f)
    



In [15]:
c = Config()
c.projects

{}